<a href="https://colab.research.google.com/github/stratis-forge/radiomics-workflows/blob/main/IBSI2_benchmark_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Validating CERR radiomics against IBSI2 benchmarks 




## Requirements
* GNU Octave with image, statistics, and io packages
* Oct2Py
* CERR
* Python with matplotlib, numpy, and ipywidgets packages

Note: Installing these tools may incur a one-time extra runtime overhead.

## I/O
This notebook demonstrates how to run unit tests to evaluate CERR's compatibility with IBSI-2 benchmarks.
DICOM-format phantoms, scans, and segmentations provided by the [IBSI](https://github.com/theibsi/data_sets) were imported to CERR's native planC format. Preprocessing and filter parameters are input via JSON files. 

## Install dependencies
Uncomment (Ctrl+/) the following sections to install (1) Octave with required packages (2) Oct2py and (3) CERR.

##Download latest Octave compile

In [1]:
# %%capture
# ! apt-get update
# ! apt-get install libgraphicsmagick++1-dev libsuitesparse-dev libqrupdate1 \
# libfftw3-3 gnuplot zsh openjdk-8-jdk
# ! cd /usr/lib/x86_64-linux-gnu/ && ln -s libhdf5_serial.so.103 libhdf5_serial.so.100 \
# && ln -s libreadline.so.8 libreadline.so.7
# !apt-get install gfortran-7

In [2]:
# #Download latest compiled octave package 
# def get_octave(root_path):
#   os.chdir(root_path)
#   with urllib.request.urlopen("https://api.github.com/repos/cerr/octave-colab/releases/latest") as url:
#       data = json.loads(url.read().decode())
#   fname = data['assets'][0]['name']
#   requrl = data['assets'][0]['browser_download_url']
#   urllib.request.urlretrieve(requrl, fname)
#   # Unzip, identify octave folder name
#   !tar xf {fname}
#   top_folder = !tar tf {fname} | head -1
#   octave_folder = top_folder[0][:-1]
#   octave_path = os.path.join(root_path,octave_folder)
#   return octave_path

In [3]:
# # Set path to Octave exectuable
# import os, urllib.request, json
# octave_path = get_octave('/content')
# os.environ['OCTAVE_EXECUTABLE'] = octave_path + '/bin/octave-cli'  #Replace with OCTAVE_EXECUTABLE path
# os.environ['PATH'] = octave_path + '/bin:' + os.environ['PATH']    #Replace with Octave path

In [4]:
# %%capture
# ! pip3 install octave_kernel
# ! pip3 install oct2py

## Download CERR 

In [5]:
%%capture
# !git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git  
# !cd /content/CERR
# !git checkout 18d4821decf4ffd45a6474c4d9f6dc85840af682
# !cd /content

## Run unit tests for IBSI-2 compatibility

In [6]:
# %load_ext oct2py.ipython
# from oct2py import octave

In [7]:
# %%capture
# %%octave
# pkg load statistics
# pkg load image
# pkg load io

In [8]:
# %%capture
# %%octave
# #Add CERR to path
# curr_dir = pwd()
# cd('/content/CERR')
# addToPath2(pwd)
# cd(curr_dir)

In [9]:
%%octave

# Create output dir
result_dir = '/content/test'
mkdir(result_dir)

# Specify IBSI-2 configuration to evaluate
config = '3b'; #Replace with desired config

# Run IBSI test with latest version of CERR
#This test uses phantom images distributed by
#the IBSI at https://github.com/theibsi/data_sets
#Preprocessing & filter parameters are supplied via JSON 
#(E.g. CERR/Unit_Testing/settings_for_comparisons/IBSIPhase2-1ID3a1.json)  
runIBSI2benchmarkFilters(result_dir,config);

## Compare with validated CERR benchmark


In [10]:
%%octave

function scan3M = getResponseMap(filename)

  #Import to CERR  
  planC = nii2cerr(filename,'CT',[],0);
  planC = updatePlanFields(planC);
  planC = quality_assure_planC(filename, planC);
  indexS = planC{end};

  scan3M = double(getScanArray(1,planC));
  scan3M = scan3M - double(planC{indexS.scan}(1).scanInfo(1).CTOffset);

end

In [11]:
%%capture
%%octave

#Load response maps
benchmark_filename = '/content/CERR/Unit_Testing/data_for_cerr_tests/IBSI2_synthetic_phantoms/Results/3b1.nii';
benchmark_map = getResponseMap(benchmark_filename);

test_filename = '/content/test/3b1.nii';
test_map = getResponseMap(test_filename);

In [12]:
%%octave

#Display max difference
diff_map = benchmark_map - test_map;

maxDiff = max(diff_map(:));
sprintf('Max diff: %0.5g',maxDiff)

## Compare scalar features extracted from lung CT phantom

In [13]:
%%octave

phase = 2; #IBSI validation phase 
runIBSI2benchmarkStatistics(result_dir,phase);

In [14]:
import pandas as pd
output_statistics_file = '/content/test/IBSIphase2-2.csv'
df = pd.read_csv (output_statistics_file)
df_subset = df.iloc[:, : 8]
print(df_subset.head())

       family                                       feature_name  \
0  Diagnostic       Number of voxels in ROI before interpolation   
1  Diagnostic  Number of voxels in intensity ROI-mask after i...   
2  Diagnostic  Mean intensity in intensity ROI-mask after int...   
3  Diagnostic  Max intensity in intensity ROI-mask after inte...   
4  Diagnostic  Min intensity in intensity ROI-mask after inte...   

                  feature_tag ibsi_identifier          1.A          1.B  \
0                diag_n_voxel               0  125256.0000  125256.0000   
1   diag_n_voxel_interp_reseg               1  125238.0000  357802.0000   
2  diag_mean_int_interp_reseg               2     -46.9756     -46.4075   
3   diag_max_int_interp_reseg               3     377.0000     377.0000   
4   diag_min_int_interp_reseg               4   -1000.0000    -997.0000   

           2.A          2.B  
0  125256.0000  125256.0000  
1  125238.0000  357802.0000  
2     -46.9756     -46.4075  
3     377.0000     3